In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install yfinance

In [5]:
# --- 单元格 2: 下载 BTC 并生成适配文件 ---

import yfinance as yf
import pandas as pd
import numpy as np
import datetime

print("--- 开始处理流程 ---")

# 1. 设置参数
TICKER = "BTC-USD"
START_DATE = "2018-01-01"
END_DATE = datetime.datetime.now().strftime('%Y-%m-%d')

# 2. 下载数据
print(f"正在下载 {TICKER} ({START_DATE} ~ {END_DATE})...")
try:
    df = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)
    
    # 处理 yfinance 可能返回的多级索引问题
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
        
    # 重置索引，让 Date 变成一列
    df = df.reset_index()
    
    print(f"下载成功，原始形状: {df.shape}")
    
except Exception as e:
    print(f"下载失败: {e}")
    print("请检查：1. 右侧 Settings -> Internet 是否开启？ 2. 网络连接是否正常？")
    raise

# 3. 构建核心特征 (P系列, V系列)
# 我们将 BTC 的真实数据映射到题目要求的 P (Price) 和 V (Volume)
df_adapted = pd.DataFrame()

# 基础信息
df_adapted['date_id'] = range(len(df)) # 简单的 0, 1, 2... ID

# 价格映射 (P系列)
df_adapted['P1'] = df['Close'] # 收盘价作为 P1
df_adapted['P2'] = df['Open']
df_adapted['P3'] = df['High']
df_adapted['P4'] = df['Low']

# 成交量映射 (V系列)
df_adapted['V1'] = df['Volume']

# 4. 构建目标变量 (Target)
# 题目要求预测 'market_forward_excess_returns'
# 我们用 BTC 的 "下一日收益率" 来代替
# 公式: (明天收盘 - 今天收盘) / 今天收盘
df_adapted['daily_return'] = df['Close'].pct_change()
df_adapted['market_forward_excess_returns'] = df_adapted['daily_return'].shift(-1)

# 原题目还有 'risk_free_rate'，我们设为 0 (假设无风险利率对纯币圈策略影响可忽略)
df_adapted['risk_free_rate'] = 0.0

# 5. 创建“占位符”特征 (为了兼容旧代码)
# 旧代码可能会用到 S, E, I, M, D 系列以及其他 V, P 列
# 我们将它们全部填充为 0。LGBM 训练时会自动忽略这些无信息的列。

# 定义需要伪造的列前缀
dummy_prefixes = ['D', 'E', 'I', 'M', 'S']
# 我们假设每种至少有 1 个 (例如 E1, S1...)，这就足够骗过 startswith 检查了
for prefix in dummy_prefixes:
    df_adapted[f'{prefix}1'] = 0.0

# 6. 清理数据
# 删除因为 shift 产生的最后一行 NaN
df_adapted = df_adapted.dropna()

# 7. 拆分 Train 和 Test
# 我们保留最后 180 天作为 "Test" (模仿 Kaggle 的 Public LB 长度)
test_days = 180
train_btc = df_adapted.iloc[:-test_days].copy()
test_btc = df_adapted.iloc[-test_days:].copy()

# Test 集必须有 'is_scored' 列 (旧代码需要)
test_btc['is_scored'] = True

# 8. 保存文件
train_filename = 'btc_train_adapted.csv'
test_filename = 'btc_test_adapted.csv'

train_btc.to_csv(train_filename, index=False)
test_btc.to_csv(test_filename, index=False)

print("\n" + "="*40)
print("✅ 文件生成完毕！")
print("="*40)
print(f"1. 训练集: {train_filename} | 形状: {train_btc.shape}")
print(f"2. 测试集: {test_filename}  | 形状: {test_btc.shape}")
print("\n现在，请回到您的主代码 (Final Model Notebook)，将文件路径修改为这两个文件名即可。")

# --- 单元格 3: 检查数据 ---
print("--- 训练集预览 ---")
print(pd.read_csv('btc_train_adapted.csv').head(3))

print("\n--- 包含了哪些列？ ---")
print(pd.read_csv('btc_train_adapted.csv').columns.tolist())

--- 开始处理流程 ---
正在下载 BTC-USD (2018-01-01 ~ 2025-11-27)...
下载成功，原始形状: (2887, 6)

✅ 文件生成完毕！
1. 训练集: btc_train_adapted.csv | 形状: (2705, 14)
2. 测试集: btc_test_adapted.csv  | 形状: (180, 15)

现在，请回到您的主代码 (Final Model Notebook)，将文件路径修改为这两个文件名即可。
--- 训练集预览 ---
   date_id            P1            P2            P3            P4  \
0        1  14982.099609  13625.000000  15444.599609  13163.599609   
1        2  15201.000000  14978.200195  15572.799805  14844.500000   
2        3  15599.200195  15270.700195  15739.700195  14522.200195   

            V1  daily_return  market_forward_excess_returns  risk_free_rate  \
0  16846600192      0.097011                       0.014611             0.0   
1  16871900160      0.014611                       0.026196             0.0   
2  21783199744      0.026196                       0.117333             0.0   

    D1   E1   I1   M1   S1  
0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  

--- 包含了哪些列？ ---
['date_id', 'P1', '